In [1]:
import sklearn.metrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from collections import OrderedDict
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import numpy as np
import pandas as pd



In [2]:
# 오토인코더 모듈 정의
class Autoencoder(nn.Module):
    def __init__(self, X_train):
        super(Autoencoder, self).__init__()
        data_size = X_train.size(1)
        
        
        self.encoder = nn.Sequential( 
            nn.Linear(data_size, 32), 
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),   
        )
        self.decoder = nn.Sequential(
            nn.Linear(4, 8), 
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, data_size),       
        )

    def predict(self, x):
        encoded = self.encoder(x) 
        decoded = self.decoder(encoded) 
        return decoded
    
    def fit (self, epochs,loader, criterion, optimizer):
        for epoch in range(epochs):
            for step , (x, label) in enumerate(loader):
                y = x
            
                encoded =  self.encoder(x)
                decoded =  self.decoder(encoded)
                loss = criterion(decoded , y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))

In [3]:
def AE(X_train, Y_train, X_test, epochs):
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            
            epochs : type : int
            
        @Output
            red_X_test : type : array
            
    '''
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test= torch.Tensor(X_test.values)
    
     # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X,Y)
    loader= DataLoader(dataset, batch_size=1, shuffle = True)
    
    #신경망 생성하기 
    model = Autoencoder(X)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)
    criterion = nn.MSELoss()
    model.fit(epochs, loader, criterion, optimizer)
    
    
    #학습되 모델 저장
    torch.save(model, "AE.pt")
    
    pred_X_test = model.predict(X_test)
    red_X_test = pred_X_test - X_test
    red_X_test = red_X_test.detach().numpy()
    
    red_X_test = pred_X_test - X_test
    return {"red_X_test": red_X_test}

In [4]:
df = pd.read_csv("RMS_bearing.csv")
x = df.iloc[:,[0,1,2]]
y = df.iloc[:,[3]]

X_train,X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=33)

In [5]:
red_X_test = AE(X_train, y_train, X_test, 10)

0epoch // loss =9.813097858568653e-05
tensor([[0.0750, 0.0939, 0.0960],
        [0.0779, 0.0949, 0.0989],
        [0.0838, 0.0935, 0.0992],
        [0.1650, 0.0972, 0.1013],
        [0.1550, 0.0972, 0.1016],
        [0.0892, 0.0931, 0.1002],
        [0.0783, 0.0954, 0.1004],
        [0.0762, 0.0935, 0.0982],
        [0.1641, 0.1016, 0.1056],
        [0.0901, 0.0923, 0.0981],
        [0.0778, 0.0928, 0.0986],
        [0.0774, 0.0943, 0.1000],
        [0.0763, 0.0935, 0.0968],
        [0.0790, 0.0959, 0.1020],
        [0.0791, 0.0934, 0.0991],
        [0.0785, 0.0953, 0.1075],
        [0.0774, 0.0944, 0.1044],
        [0.1102, 0.0970, 0.1025],
        [0.0778, 0.0953, 0.1053],
        [0.1035, 0.0988, 0.1035],
        [0.1351, 0.0977, 0.1007],
        [0.0762, 0.0955, 0.1010],
        [0.0781, 0.0952, 0.1011],
        [0.0775, 0.0934, 0.0993],
        [0.0896, 0.0925, 0.0973],
        [0.0984, 0.0935, 0.0996],
        [0.0765, 0.0955, 0.1032],
        [0.0770, 0.0937, 0.1014],
        [0

In [6]:
red_X_test

{'red_X_test': tensor([[ 1.8816e-03, -1.4204e-03,  1.7323e-03],
         [ 1.8308e-03, -1.7169e-03, -6.0336e-04],
         [ 1.2941e-03,  9.6158e-04,  1.4715e-04],
         [-2.8665e-03,  1.5013e-02,  1.3314e-02],
         [-2.2101e-03,  1.2838e-02,  1.1218e-02],
         [ 1.0445e-03,  2.5366e-03,  1.7360e-04],
         [ 2.0955e-03, -2.0419e-03, -1.9899e-03],
         [ 1.6906e-03, -7.2974e-04, -3.0084e-04],
         [-1.5312e-03,  1.0712e-02,  9.0894e-03],
         [ 5.7463e-04,  3.3803e-03,  2.3682e-03],
         [ 1.4858e-03,  2.3755e-04, -3.9873e-04],
         [ 1.9279e-03, -1.2121e-03, -1.7727e-03],
         [ 1.6006e-03, -7.8183e-04,  1.0951e-03],
         [ 2.3723e-03, -2.3909e-03, -3.4269e-03],
         [ 1.5600e-03,  2.5369e-05, -6.5101e-04],
         [ 3.0993e-03, -1.7535e-03, -8.8509e-03],
         [ 2.5783e-03, -1.1539e-03, -6.0834e-03],
         [ 6.7476e-04,  3.3311e-03,  1.9402e-03],
         [ 2.8202e-03, -1.9375e-03, -6.8397e-03],
         [ 1.5033e-03,  1.7354e-04, 

In [7]:
AE= torch.load("AE.pt")
AE.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=4, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [8]:
X_test = torch.Tensor(X_test.values)
pred_y_test = AE.predict(X_test)

print(pred_y_test - X_test)

tensor([[ 1.8816e-03, -1.4204e-03,  1.7323e-03],
        [ 1.8308e-03, -1.7169e-03, -6.0336e-04],
        [ 1.2941e-03,  9.6158e-04,  1.4715e-04],
        [-2.8665e-03,  1.5013e-02,  1.3314e-02],
        [-2.2101e-03,  1.2838e-02,  1.1218e-02],
        [ 1.0445e-03,  2.5366e-03,  1.7360e-04],
        [ 2.0955e-03, -2.0419e-03, -1.9899e-03],
        [ 1.6906e-03, -7.2974e-04, -3.0084e-04],
        [-1.5312e-03,  1.0712e-02,  9.0894e-03],
        [ 5.7463e-04,  3.3803e-03,  2.3682e-03],
        [ 1.4858e-03,  2.3755e-04, -3.9873e-04],
        [ 1.9279e-03, -1.2121e-03, -1.7727e-03],
        [ 1.6006e-03, -7.8183e-04,  1.0951e-03],
        [ 2.3723e-03, -2.3909e-03, -3.4269e-03],
        [ 1.5600e-03,  2.5369e-05, -6.5101e-04],
        [ 3.0993e-03, -1.7535e-03, -8.8509e-03],
        [ 2.5783e-03, -1.1539e-03, -6.0834e-03],
        [ 6.7476e-04,  3.3311e-03,  1.9402e-03],
        [ 2.8202e-03, -1.9375e-03, -6.8397e-03],
        [ 1.5033e-03,  1.7354e-04, -2.2271e-04],
        [-1.0390e-03